In [3]:
import re
import numpy as np
import pandas as pd
# import contractions
import inflect
# import nltk
# from nltk.tokenize import word_tokenize
# from nltk.corpus import stopwords
# from bs4 import BeautifulSoup

#nltk.download()
# from nltk import WordNetLemmatizer
# from tqdm import tqdm
# from tqdm.auto import tqdm
# tqdm.pandas()

from sklearn.preprocessing import MinMaxScaler
from math import log
import json
import matplotlib.pyplot as plt
import pickle

import os
import seaborn as sns
import scipy
import random
from datetime import datetime
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error

import warnings
warnings.filterwarnings("ignore")

In [ ]:
!pip install word2number

In [ ]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 14.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=2811628 sha256=488534420489bf0c147bb83f9d9efa5322faaf07e507b6c91cb4b9ab47cc3d2f
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [ ]:
from surprise import SVD, KNNBasic, KNNWithMeans, KNNBaseline, NMF
from surprise import dump
from surprise import Dataset
from surprise import Reader
from surprise import accuracy
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split

from collections import defaultdict

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 언젠가 gpu 사용할때

# import torch

# '''
# if torch가 할당이 안되면요...
# 1) anaconda prompt 관리자권한 실행
# 2) conda activate janetorch(내가 쓰는 가상환경임)
# 2) conda install pytorch torchvision torchaudio pytorch-cuda=11.7 -c pytorch -c nvidia
# --> 내 버전 cuda11.7임....
# --> version : pytorch 2.0.1  pytorch-mutex 1.0  torchvision 0.15.2  py3.10_cuda 11.7  cudnn8.0  py310-cu117
# '''


# device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
# # GPU 사용 가능 -> True, GPU 사용 불가 -> False
# print(torch.cuda.is_available())

# print(torch.__version__)
# # # GPU 여러개를 지정하고 싶은 경우는 아래 형태로 사용
# # os.environ["CUDA_VISIBLE_DEVICES"]="0,1,2"

## 00. open data

In [5]:
path = '/content/drive/MyDrive/00.multi_emtions/03.models'

# 영화 정보가 담긴 데이터
# rating_df = pd.read_csv(os.path.join(path, 'ratings.csv'), encoding='utf-8')

In [6]:
with open(os.path.join(path, 'movies_train.pkl'), 'rb') as f:
    train_set = pickle.load(f)
with open(os.path.join(path,'movies_test.pkl'), 'rb') as f:
    test_set = pickle.load(f)
with open(os.path.join(path,'movies_validation.pkl'), 'rb') as f:
    valid_set = pickle.load(f)

### 데이터 설명
- userID : 유저 아이디
- movieID : 영화 아이디
- rating : 유저가 영화에 남긴 평점
- compound : VADER 기준 sentiment score로 -1~1 범위(float)
- senti_score : NeMF를 위한 compound 기반 1~3의 sentiment score(compound -0.05 미만:1  /  -0.05~0.05:2  /  0.05 초과:3)
- NeMFRating : rating기준 1~3 정규화(rating 1,2 : 1  /  3 : 3  /  4,5 : 3)
- SBMFRating : SBMF를 위한 compound를 1~5점으로 정규화(min-max 정규화)
- anger~trust : 8 emotions based on NRCdictionary&LIWC
- negative, positive : sentiment score based on NRCdictionary&LIWC

In [ ]:
len(train_set)

2250979

In [ ]:
len(test_set)

643138

In [ ]:
len(valid_set)

321569

In [7]:
seed_num = 42

# 00-1. MF

In [6]:
import numpy as np

# 위에서 구한 MF를 SGD로 학습하는 것을 class 형태로 만듦

class MF():

    def __init__(self, R, K, learning_rate, regularization, iterations):
        """
        Arguments
        - R (ndarray)             : 유저 아이템 매트릭스
        - K (int)                 : latent factor 차원
        - learning_rate (float)   : learning rate
        - regularization (float)  : regularization parameter
        """
        self.R = R
        self.num_users, self.num_items = R.shape
        self.K = K
        self.learning_rate = learning_rate
        self.regularization = regularization
        self.iterations = iterations

    def train(self):

        # 유저, 아이템 잠재 요인 행렬 초기화
        self.P = np.random.normal(scale=1./self.K, size=(self.num_users, self.K))
        self.Q = np.random.normal(scale=1./self.K, size=(self.num_items, self.K))

        # 유저, 아이템, 글로벌 bias 초기화
        self.b_u = np.zeros(self.num_users)
        self.b_i = np.zeros(self.num_items)
        self.b = np.mean(self.R[np.where(self.R != 0)])

        # 학습 데이터 생성
        # self.samples = [
        #     (i, j, self.R[i, j])
        #     for i in range(self.num_users)
        #     for j in range(self.num_items)
        #     if self.R[i, j] > 0
        # ]

        self.samples = self.R

        # iteration 개수만큼 전체 데이터에 대해서 SGD 수행
        training_process = []
        for i in tqdm(range(self.iterations)):
            np.random.shuffle(self.samples)
            self.sgd()
            rmse = self.rmse()
            mae = self.mae() # mae 필요함
            training_process.append((i, rmse, mae))

            # iteration 10의 배수마다 rmse 출력
            # if (i+1) % 10 == 0:
            #     print("Iteration: %d, error = %.4f" % (i+1, rmse))

            # 모든 iteration의 rmse 출력
            print("Iteration: %d, rmse = %.4f, mae = %.4f" % (i, rmse, mae))

        return pd.DataFrame(training_process, columns = ['iteration', 'rmse', 'mae'])

    def rmse(self):
        """
        전체 학습 데이터에 대한 rmse 계산
        """
        ut, it = self.R.nonzero()
        predicted_R = self.full_matrix()
        error = []
        for x, y in zip(ut, it):
            error.append(pow(R[x, y] - predicted_R[x, y], 2))
        return np.sqrt(np.asarray(error).mean())

    def mae(self):
      # mae 계산 함수 추가
        ut, it = self.R.nonzero()
        predicted_R = self.full_matrix()
        error = []
        for x, y in zip(ut, it):
            error.append(np.abs(self.R[x, y] - predicted_R[x, y]))
        return np.asarray(error).mean()


    def sgd(self):
        """
        sgd로 파라미터 업데이트가 되는 함수
        """
        for i, j, r in self.samples:
            # 실제 rating과 predicted rating을 구하고 error를 구함
            prediction = self.get_rating(i, j)
            e = (r - prediction)

            # bias 업데이트
            self.b_u[i] += self.learning_rate * (e - self.regularization * self.b_u[i])
            self.b_i[j] += self.learning_rate * (e - self.regularization * self.b_i[j])

            # 유저, 아이템 잠재 요인 행렬 업데이트
            self.P[i, :] += self.learning_rate * (e * self.Q[j, :] - self.regularization * self.P[i,:])
            self.Q[j, :] += self.learning_rate * (e * self.P[i, :] - self.regularization * self.Q[j,:])

    def get_rating(self, i, j):
        """
        유저 i, 아이템 j에 대한 예측 평점
        """
        prediction = self.b + self.b_u[i] + self.b_i[j] + self.P[i, :].dot(self.Q[j, :].T)
        return prediction

    def full_matrix(self):
        """
        예측된 유저 아이템 매트릭스를 계산 Rhat
        """
        return self.b + self.b_u[:,np.newaxis] + self.b_i[np.newaxis:,] + self.P.dot(self.Q.T)

    def test_mae_rmse(self, test_set):
        ut, it = test_set.nonzero()
        predicted_R = self.full_matrix()
        error_mae = []
        error_rmse = []
        for x, y in zip(ut, it):
            error_mae.append(np.abs(test_set[x, y] - predicted_R[x, y]))
            error_rmse.append(pow(test_set[x, y] - predicted_R[x, y], 2))
        test_mae = np.asarray(error_mae).mean()
        test_rmse = np.sqrt(np.asarray(error_rmse).mean())
        return test_mae, test_rmse

In [8]:
# 학습데이터, test 데이터 나누기
train_set = train_set.append(valid_set)
test_set

,userID,movieID,rating,compound,senti_score,NeMFRating,SBMFRating,anger,anticipation,disgust,fear,joy,sadness,surprise,trust,negative,positive
3158911,276374,48067,5.0,0.6249,3,3,4,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1704440,149783,541,5.0,0.6369,3,3,4,0.00,0.00,0.00,0.00,50.00,0.00,0.00,0.00,0.00,50.00
2971340,260113,47119,5.0,0.9600,3,3,5,0.00,12.00,0.00,0.00,16.00,0.00,16.00,16.00,0.00,28.00
1213057,106354,18996,1.0,0.9870,3,1,5,1.48,9.63,2.96,5.19,5.93,2.22,5.19,11.85,5.93,13.33
2806975,245581,28162,4.0,0.8625,3,3,5,0.00,23.08,0.00,0.00,7.69,0.00,7.69,7.69,0.00,23.08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30437,2626,32298,5.0,0.9595,3,3,5,8.70,4.35,2.90,8.70,10.14,4.35,4.35,1.45,10.14,20.29
653889,57185,8722,5.0,0.9590,3,3,5,0.00,8.00,0.00,8.00,10.00,4.00,2.00,8.00,4.00,10.00
2224435,194822,28068,5.0,0.4215,3,3,4,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1501535,131790,49106,5.0,0.4404,3,3,4,0.00,100.00,0.00,0.00,100.00,0.00,100.00,100.00,0.00,100.00


In [1]:
# 학습데이터, 하이퍼파라미터 initialize

# 유저 아이템의 Rating 매트릭스(터진다)
user_item_matrix = train_set.pivot_table('rating', 'userID', 'movieID').fillna(0)

# train_set으로 학습하기
R = user_item_matrix.to_numpy()
# 메트

mf = MF(R, K=20, learning_rate=0.01, regularization=0.2, iterations=30)


NameError: ignored

In [ ]:
# MF 모델 학습
train_result_df = mf.train()

In [ ]:
# 예측 매트릭스
# 예측된 유저 아이템 매트릭스를 계산 Rhat
# 그럼 5점 넘으면 안되는거 아닌가
mf.full_matrix()

In [ ]:
# test 해보기
# iteration에 따른 training loss

x = train_result_df.iteration.values + 1
y = train_result_df.rmse.values
plt.figure(figsize=((8,4)))
plt.plot(x, y)
plt.xticks(x, x)
plt.xlabel("Iterations")
plt.ylabel("RMSE")
plt.grid(axis="y")

In [17]:
# Calculate test MAE and RMSE
test_mae, test_rmse = mf.test_mae_rmse(test_set)

print("Test MAE:", test_mae)
print("Test RMSE:", test_rmse)